In [ ]:
using QuantumOptics
using DifferentialEquations
import Base: ==


###### FROM MODULE #####

const QuantumState = Union{Ket, DenseOperator}

type State{T<:QuantumState}
    quantum::T
    classical::Vector{Complex128}
end

Base.length(state::State) = length(state.quantum) + length(state.classical)
Base.copy(state::State) = State(copy(state.quantum), copy(state.classical))

function =={T<:QuantumState}(a::State{T}, b::State{T})
    samebases(a.quantum, b.quantum) &&
    length(a.classical)==length(b.classical) &&
    (a.classical==b.classical) &&
    (a.quantum==b.quantum)
end

operators.expect(op, state::State) = expect(op, state.quantum)

# Modified master equation solver
function master_stochastic(tspan, state0::State{DenseOperator}, fquantum, fdeterm, fstoch;
                solver=EulerHeun(),
                rates::Union{Vector{Float64}, Matrix{Float64}, Void}=nothing,
                fout::Union{Function,Void}=nothing,
                tmp::DenseOperator=copy(state0.quantum))
    tspan_ = convert(Vector{Float64}, tspan)
    function dmaster_(t, state::State{DenseOperator}, dstate::State{DenseOperator})
        dmaster_h_determ(t, state, fquantum, fdeterm, rates, dstate, tmp)
    end
    x0 = Vector{Complex128}(length(state0))
    recast!(state0, x0)
    state = copy(state0)
    dstate = copy(state0)
    integrate(tspan_, dmaster_, fstoch, x0, state, dstate, solver)
end

# Modified integrate function
function integrate(tspan_, dmaster_determ, fstoch, x0, state, dstate, solver)
  function df_(t, x::Vector{Complex128}, dx::Vector{Complex128})
   recast!(x, state)
   recast!(dx, dstate)
   dmaster_determ(t, state, dstate)
   recast!(dstate, dx)
  end
  prob = SDEProblem(df_, fstoch, x0, (tspan_[1], tspan_[end]))
  sol = solve(prob, solver)
  rho_out = []
  tmp = State(DenseOperator(state.quantum.basis_l, state.quantum.basis_r), Vector{Complex128}(length(state.classical)))
  for i=1:length(sol.t)
    push!(rho_out, copy(recast!(sol.u[i], tmp)))
  end
  return sol.t, rho_out
end

function recast!(state::State, x::Vector{Complex128})
    N = length(state.quantum)
    copy!(x, 1, state.quantum.data, 1, N)
    copy!(x, N+1, state.classical, 1, length(state.classical))
    x
end

function recast!(x::Vector{Complex128}, state::State)
    N = length(state.quantum)
    copy!(state.quantum.data, 1, x, 1, N)
    copy!(state.classical, 1, x, N+1, length(state.classical))
    state
end

# Modified dmaster
function dmaster_h_determ(t::Float64, state::State{DenseOperator}, fquantum, fdeterm, rates, dstate::State{DenseOperator}, tmp::DenseOperator)
    fquantum_(t, rho) = fquantum(t, state.quantum, state.classical)
    timeevolution.timeevolution_master.dmaster_h_dynamic(t, state.quantum, fquantum_, rates, dstate.quantum, tmp)
    fdeterm(t, state.quantum, state.classical, dstate.classical)
end


######### EXAMPLE ########

# Parameters
Nc = 16
κ = 1.0
g = 0.5κ
γ = 2κ
Δa = -0.5γ
Δc = 0.0
η = 1.0
ξ = 0.1 # Noise strength
m = 3.33
tlist = [0:0.1:100;]

# Basis
bc = FockBasis(Nc)
ba = SpinBasis(1//2)

a = destroy(bc) ⊗ one(ba)
aᵗ = create(bc) ⊗ one(ba)
σ = one(bc) ⊗ sigmam(ba)
σᵗ = one(bc) ⊗ sigmap(ba)

H0 = -Δc*aᵗ*a - Δa*σᵗ*σ + η*(a + aᵗ)
Hx = g*(aᵗ*σ + a*σᵗ) # ∝ cos(x)
J = [sqrt(κ)*a, sqrt(γ)*σ]
Jdagger = dagger.(J)

function fquantum(t, ψ, u)
  H0 + Hx*cos(real(u[1])), J, Jdagger
end

atσ = aᵗ*σ
function fdeterm(t, ψ, u, du)
  du[1] = u[2]/m
  du[2] = 2g*sin(real(u[1]))*real(expect(atσ, ψ))
end

function fstoch(t, u, du)
  du[end] = ξ
end

x0 = sqrt(2)
p0 = 4
u0 = Complex128[x0, p0]
ρ0 = State(dm(fockstate(bc, 0) ⊗ spindown(ba)), u0)

tout, ρt = master_stochastic(tlist, ρ0, fquantum, fdeterm, fstoch; solver=SRI())

x = [r.classical[1] for r=ρt]
p = [r.classical[2] for r=ρt]

pe = expect(σᵗ*σ, ρt)
pg = expect(σ*σᵗ, ρt)

n = expect(aᵗ*a, ρt)

using PyPlot
subplot(221)
plot(tout, p)

subplot(222)
plot(tout, x)

subplot(223)
plot(tout, pe)

subplot(224)
plot(tout, n)